In [40]:
import numpy as np
import pandas as pd
import requests

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [3]:
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [4]:
user_question = "I just discovered the course. Can I still join it?"

In [17]:
encoded_question = model.encode(user_question)
encoded_question[0]

0.07822265

In [7]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
filtering_key = "machine-learning-zoomcamp"

In [10]:
documents = [doc for doc in documents if doc["course"] == filtering_key]
len(documents)

375

In [11]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [14]:
embeddings = []
for doc in tqdm(documents):
    question = doc["question"]
    text = doc["text"]
    qa_text = f"{question} {text}"
    embeddings.append(model.encode(qa_text))

100%|██████████| 375/375 [00:41<00:00,  9.12it/s]


In [15]:
X = np.array(embeddings)

In [16]:
X.shape

(375, 768)

In [18]:
scores = X.dot(encoded_question)

In [19]:
scores.max()

0.65065753

In [20]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(encoded_question, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [22]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [25]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [27]:
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)

In [38]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for doc in tqdm(ground_truth):
        curr_question = doc["question"]
        doc_id = doc["document"]
        encoded_question = model.encode(curr_question)
        results = search_function(encoded_question)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [39]:
evaluate(ground_truth, search_function=lambda vec: search_engine.search(vec, 5))

100%|██████████| 1830/1830 [04:21<00:00,  6.99it/s]


{'hit_rate': 0.9398907103825137}

In [42]:
es_client = Elasticsearch('http://elasticsearch:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [44]:
for doc, embedding in tqdm(zip(documents, embeddings)):
    doc["embedding"] = embedding

375it [00:00, 236912.79it/s]


In [45]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:02<00:00, 182.44it/s]


In [55]:
def elastic_search_knn(field, vector, print_hits=False):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        if print_hits:
            print(hit)
        result_docs.append(hit['_source'])

    return result_docs

In [56]:
_ = elastic_search_knn("embedding", encoded_question, print_hits=True)

{'_index': 'course-questions', '_id': '030ByJABIOU-Kp90BFEG', '_score': 0.7454634, '_source': {'question': 'I just joined. What should I do next? How can I access course materials?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions', 'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you can just use this link: http://mlzoomcamp.com/#syllabus', 'id': '0a278fb2'}}
{'_index': 'course-questions', '_id': 'w30ByJABIOU-Kp90A1GH', '_score': 0.74376416, '_source': {'question': 'Is it going to be live? When?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions'

In [57]:
evaluate(ground_truth, search_function=lambda vec: elastic_search_knn("embedding", vec))

100%|██████████| 1830/1830 [02:14<00:00, 13.56it/s]


{'hit_rate': 0.9398907103825137}